In [1]:
import cortex
import nibabel
import nipype
import numpy as np
from matplotlib import colors



In [2]:
currsub = 'sub-8032ctbs'
datadir = '/mnt/data0/tbs_opiate/data/'
anatdata = datadir+currsub+'/ses-preTMS/derivatives/'+currsub+'_ses-preTMS_T1w.anat/T1_biascorr_brain.nii.gz'
funcdata = datadir+currsub+'/ses-preTMS/derivatives/denoised_pain/fs_spm.feat/example_func.nii.gz'
hiktdata = datadir+currsub+'/ses-preTMS/derivatives/denoised_pain/hikts_anat_aligned.nii.gz'

pain_map = datadir+currsub+'/ses-preTMS/derivatives/denoised_pain/pain_medn_nat.feat/stats/zstat1.nii.gz'
hikts_comps = datadir+currsub+'/ses-preTMS/derivatives/denoised_pain/pain_mefcz_nat.nii.gz'


In [5]:
cortex.freesurfer.import_subj(currsub)

created by dowdlelt on Wed Oct 25 12:23:58 2017

created by dowdlelt on Wed Oct 25 13:07:59 2017

created by dowdlelt on Wed Oct 25 12:23:58 2017

created by dowdlelt on Wed Oct 25 13:08:09 2017

created by dowdlelt on Wed Oct 25 12:23:58 2017

created by dowdlelt on Wed Oct 25 12:23:58 2017

created by dowdlelt on Wed Oct 25 13:07:59 2017

created by dowdlelt on Wed Oct 25 13:08:09 2017

created by dowdlelt on Wed Oct 25 12:24:11 2017

created by dowdlelt on Wed Oct 25 12:24:12 2017



In [6]:
cortex.freesurfer.import_flat(currsub, 'full')

created by dowdlelt on Wed Oct 25 12:23:58 2017

saving to /home/dowdlelt/anaconda2/share/pycortex/db/sub-8032ctbs/surfaces/flat_lh.gii
created by dowdlelt on Wed Oct 25 12:23:58 2017

saving to /home/dowdlelt/anaconda2/share/pycortex/db/sub-8032ctbs/surfaces/flat_rh.gii


In [7]:
cortex.align.automatic(currsub, 'fullhead',anatdata)

Generating brainmask anatomical...
Brain masking anatomical...
Generating whitematter anatomical...
Segmenting the brain...
FLIRT pre-alignment
Running BBR
Success


In [8]:
cortex.align.automatic(currsub,'funcspace',funcdata)

FLIRT pre-alignment
Running BBR
Success


In [3]:
hikt_pain = nibabel.load(hiktdata)
comp_zscores = nibabel.load(hikts_comps)
pain_act = nibabel.load(pain_map)

In [4]:
act_vol = cortex.Volume(pain_act.get_data().T.astype(float), currsub,'funcspace', priority=1)
print('act_vol shape is' ,act_vol.shape)

act_thresh = (pain_act.get_data().T.astype(float))
act_thresh[np.absolute(act_thresh)<3.1] = np.nan
act_thresh_vol = cortex.Volume(act_thresh, currsub, 'funcspace', priority=2)
print('act_thresh shape is' ,act_thresh.shape)


hikt_vol = cortex.Volume(hikt_pain.get_data().T.astype(float), currsub,'funcspace', priority=3)
print('hikt_vol shape is' ,hikt_vol.shape)

hikt_thresh = hikt_pain.get_data().T.astype(float)
pain_swapped = act_thresh.reshape(70, 96, 69)

hikt_thresh[:,np.isnan(pain_swapped)] = np.nan


hikt_thresh_vol = cortex.Volume(hikt_thresh, currsub, 'funcspace', priority=5)
kappa_comps_vol = cortex.Volume(comp_zscores.get_data().T.astype(float), currsub,'funcspace', priority=4)

volumes = {
    'Pain Map': act_vol,
    'Pain Map (Z>3.1)': act_thresh_vol,
    'Timeseries': hikt_vol,
    'High Kappa Components': kappa_comps_vol,
    'timeseries, masked': hikt_thresh_vol
}

('act_vol shape is', (70, 96, 69))
('act_thresh shape is', (70, 96, 69))
('hikt_vol shape is', (70, 96, 69))


In [5]:
print(hikt_thresh.shape)
print(hikt_pain.shape)
print(pain_act.shape)
print(act_thresh.shape)
print(act_vol.shape)
print(hikt_vol.shape)

(445, 70, 96, 69)
(69, 96, 70, 445)
(69, 96, 70)
(70, 96, 69)
(70, 96, 69)
(70, 96, 69)


In [6]:
cortex.webshow(data=volumes)

Started server on port 51790


<JS: window.viewer>

In [14]:
cortex.webshow((comp_zscores.get_data().T.astype(float),currsub,'funcspace'))

Started server on port 57668


<JS: window.viewer>

Stopping server


In [11]:
cortex.webshow((hikt_pain.get_data().T.astype(float),currsub,'funcspace'))

Generating new ctm file...
wm
wm
inflated
inflated
Generating a flatmap cache
Started server on port 43347


<JS: window.viewer>

In [ ]:
cortex.webshow((pain_act.get_data().T.astype(float),currsub,'funcspace'))

In [3]:
cortex.freesurfer.import_flat('fsaverage_copied', 'full')

created by greve on Thu Apr 11 16:23:42 2013

saving to /home/dowdlelt/anaconda2/share/pycortex/db/fsaverage_copied/surfaces/flat_lh.gii
created by greve on Thu Apr 11 16:23:51 2013

saving to /home/dowdlelt/anaconda2/share/pycortex/db/fsaverage_copied/surfaces/flat_rh.gii


In [8]:
group_pain = nibabel.load('/mnt/data0/tbs_opiate/models/fsl/controls/pre/flame2_21_3pt1.gfeat/cope1.feat/stats/zstat1.nii.gz')
cortex.align.automatic('fsaverage_copied', 'fsl_space','/mnt/data0/tbs_opiate/models/fsl/controls/pre/flame2_21_3pt1.gfeat/bg_image.nii.gz')

Generating brainmask anatomical...
Brain masking anatomical...
Generating whitematter anatomical...
Segmenting the brain...
FLIRT pre-alignment
Running BBR
Success


In [9]:
cortex.webshow((group_pain.get_data().T.astype(float),'fsaverage_copied','fsl_space'))

Generating new ctm file...
wm
wm
inflated
inflated
Generating a flatmap cache
Started server on port 9656


<JS: window.viewer>

Stopping server
